# Imports

In [12]:
from api.rest import QueryEngine


In [42]:

qe = QueryEngine() 
a = qe.get_calculation(band_gap_range=[1,1.5], formation_energy_range=[124,150])
for e in a:

    if len(str(e).split('@')[0])< 8:
        #print(str(c[d]).split('@')[0])
        print(str(e).split('@')[0] + '\t\t BandGap = \t' + str(round(e.band_gap,4)) + 
              ' eV\t Formation Eneergy = \t' + str(e.formation_energy)+ ' meV')
    else:
        print(str(e).split('@')[0] + '\t BandGap = \t' + str(round(e.band_gap,4)) +
              ' eV\t Formation Eneergy = \t' + str(e.formation_energy) + ' meV')

query info     band_gap_range: [1, 1.5]  formation_energy_range: [124, 150] elements: [] space_group_number: None dimension: None crystal_system: None
len all 0
Pd2S4 		 BandGap = 	1.1746 eV	 Formation Eneergy = 	148.0 meV
Ge2Se2 		 BandGap = 	1.2107 eV	 Formation Eneergy = 	133.0 meV
Pt4I8 		 BandGap = 	1.4215 eV	 Formation Eneergy = 	135.0 meV
Pd4I8 		 BandGap = 	1.1512 eV	 Formation Eneergy = 	133.0 meV
MoO3 		 BandGap = 	1.1381 eV	 Formation Eneergy = 	136.0 meV


In [43]:
c = a.last()
c.get_soap()

['Mo', 'O']


array([[ 1.5950422e-05,  2.8637054e-05, -2.5482109e-04, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 1.5111515e-02,  7.2775774e-02,  1.7874929e-01, ...,
         2.0626926e-06, -1.1114194e-05,  5.9885468e-05],
       [ 1.5436095e-02,  7.4300393e-02,  1.8286280e-01, ...,
         5.8788451e-06, -2.7299011e-05,  1.2676571e-04],
       [ 1.5505436e-02,  7.4621662e-02,  1.8370523e-01, ...,
         6.6426281e-07, -4.2106908e-06,  2.6691119e-05]], dtype=float32)

In [44]:
%cd /home/jason/dev/materialsweb
%rm -rf formation_energy
c.prep_ml_formation_energy()
%ls

/home/jason/dev/materialsweb
0.energy   12.energy  15.energy  18.energy  3.energy  6.energy  9.energy
0.poscar   12.poscar  15.poscar  18.poscar  3.poscar  6.poscar  9.poscar
10.energy  13.energy  16.energy  1.energy   4.energy  7.energy  OsZICAR
10.poscar  13.poscar  16.poscar  1.poscar   4.poscar  7.poscar  XdATCAR
11.energy  14.energy  17.energy  2.energy   5.energy  8.energy
11.poscar  14.poscar  17.poscar  2.poscar   5.poscar  8.poscar


In [46]:
c.dos.plot()

TypeError: string indices must be integers